In [1]:
print("all ok")

all ok


In [2]:
from typing import List, Sequence

from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import os

load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [3]:
model_client=OpenAIChatCompletionClient(model='gpt-4o')

In [4]:
# Note: This example uses mock tools instead of real APIs for demonstration purposes
def search_web_tool(query: str) -> str:
    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        """
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."





In [5]:
def percentage_change_tool(start: float, end: float) -> float:
    return ((end - start) / start) * 100

In [6]:
from langchain_community.utilities import GoogleSerperAPIWrapper
from autogen_ext.tools.http import HttpTool

os.environ["SERPER_API_KEY"]=os.getenv("SERPER_API_KEY")
search_tool=GoogleSerperAPIWrapper(type='search')
def search_web(query : str)-> str:
    """Search the web with the given query and provide the results."""
    try:
        result=search_tool.run(query)
        return result
    except Exception as e:
        print(f"Error occured while searching the web: {e}")
        return "No result found"

In [7]:


planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        WebSearchAgent: Searches for information
        DataAnalystAgent: Performs calculations

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

web_search_agent = AssistantAgent(
    "WebSearchAgent",
    description="An agent for searching information on the web.",
    #tools=[search_web_tool],
    tools=[search_web],
    model_client=model_client,
    system_message="""
    You are a web search agent.
    Your only tool is search_web - use it to find information.
    You make only one search call at a time.
    Once you have the results, you never do calculations based on them.
    """,
)

data_analyst_agent = AssistantAgent(
    "DataAnalystAgent",
    description="An agent for performing calculations.",
    model_client=model_client,
    tools=[percentage_change_tool],
    system_message="""
    You are a data analyst.
    Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided.
    If you have not seen the data, ask for it.
    """,
)


In [8]:
text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination


In [9]:
selector_prompt = """Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure the planner agent has assigned tasks before other agents start working.
Only select one agent.
"""


In [10]:
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,  # Allow an agent to speak multiple turns in a row.
)


In [11]:
task = """Who was the Miami Heat player with the highest points in the 2006-2007 season, 
 what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"""


In [13]:
# Use asyncio.run(...) if you are running this in a script.
await team.reset()
await Console(team.run_stream(task=task))


---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, 
 what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
To address your request, let's break down the steps required:

1. Identify the Miami Heat player with the highest points during the 2006-2007 NBA season.
2. Obtain the total rebounds for that player in the 2007-2008 season.
3. Obtain the total rebounds for that player in the 2008-2009 season.
4. Calculate the percentage change in total rebounds between the 2007-2008 and 2008-2009 seasons.

Let's delegate these tasks:

1. WebSearchAgent: Find the Miami Heat player with the highest points scored in the 2006-2007 NBA season.
2. WebSearchAgent: Search for the total rebounds of this player during the 2007-2008 NBA season.
3. WebSearchAgent: Search for the total rebounds of this player during the 2008-2009 NBA season.
4. DataA

TaskResult(messages=[TextMessage(id='cc658df9-34db-4474-9f96-582f27a040ed', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 23, 16, 42, 55, 444518, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest points in the 2006-2007 season, \n what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='c7662024-249b-4f81-842c-7343f40370f4', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=161, completion_tokens=213), metadata={}, created_at=datetime.datetime(2025, 7, 23, 16, 43, 1, 94964, tzinfo=datetime.timezone.utc), content="To address your request, let's break down the steps required:\n\n1. Identify the Miami Heat player with the highest points during the 2006-2007 NBA season.\n2. Obtain the total rebounds for that player in the 2007-2008 season.\n3. Obtain the total rebounds for that player in the 2008-2009 season.\n4. Calculate